In [106]:
import nltk
import pandas as pd
import numpy as np

In [107]:
reviews = pd.read_table(
    r"reviews.txt", header=None, encoding="utf-8"
)

reviews[0].value_counts()

0
positive    51
negative    50
Name: count, dtype: int64

In [108]:
one_hot = pd.get_dummies(reviews[0])
# Drop column B as it is now encoded
df = reviews.drop(0, axis=1)
# Join the encoded df
df = df.join(one_hot)
df = df.rename(columns={1: 'review', 'positive': 'assessment'}).drop('negative', axis=1)
df

,review,assessment
0,"Игра, в которой победить может даже злодей. От...",True
1,Обалденная игра для компании. Помогает разнооб...,True
2,"Весёлая настольная игра для компании, которая ...",True
3,Общие впечатления Игра огонь даже для взрослог...,True
4,Случайно наткнулся на эту игрушку в детском ми...,True
...,...,...
96,Ерунда полная! Половина больших бакуганов не т...,False
97,"Ужасное качество фишек , древесина (загатовка ...",False
98,"Инструкция к детской игре должна предполагать,...",False
99,"Покупали шар головоломку, намного интереснее и...",False


In [109]:
extra_reviews = pd.read_csv(r"r.csv")
new_df = extra_reviews.rename(
    columns={"Unnamed: 0": "ex_id", "recommended": "assessment"}
).drop("ex_id", axis=1)
new_df

,review,assessment
0,"Спешл эдишн дали халявно, т.к. ранее покупал о...",True
1,"Моя самая любимая игра!Игра интересна тем что,...",True
2,ИГРА ПРОСТО КАК ТЕС ОЧЕНЬ ДАЖЕ ПЕРВАЯ,True
3,"Игра просто бомба, рекомендую всем",True
4,"очень хорошая игра, если не играли то очень со...",True
...,...,...
33593,"песочница в котрой развлекаешь себя сам,в перв...",True
33594,"Спасибо за игру.Если меня спросят,когда меня т...",False
33595,У игры множество технических проблем. Постоянн...,False
33596,Не ну я конечно подазревал что игра будет мягк...,False


In [110]:
data = pd.concat([df, new_df])
data

,review,assessment
0,"Игра, в которой победить может даже злодей. От...",True
1,Обалденная игра для компании. Помогает разнооб...,True
2,"Весёлая настольная игра для компании, которая ...",True
3,Общие впечатления Игра огонь даже для взрослог...,True
4,Случайно наткнулся на эту игрушку в детском ми...,True
...,...,...
33593,"песочница в котрой развлекаешь себя сам,в перв...",True
33594,"Спасибо за игру.Если меня спросят,когда меня т...",False
33595,У игры множество технических проблем. Постоянн...,False
33596,Не ну я конечно подазревал что игра будет мягк...,False


In [111]:
data['assessment'].value_counts()

assessment
True     27295
False     6404
Name: count, dtype: int64

In [112]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
label = enc.fit_transform(data['assessment'])
label

array([1, 1, 1, ..., 0, 0, 1], dtype=int64)

In [113]:
processed = data['review'].str.lower()
processed

0        игра, в которой победить может даже злодей. от...
1        обалденная игра для компании. помогает разнооб...
2        весёлая настольная игра для компании, которая ...
3        общие впечатления игра огонь даже для взрослог...
4        случайно наткнулся на эту игрушку в детском ми...
                               ...                        
33593    песочница в котрой развлекаешь себя сам,в перв...
33594    спасибо за игру.если меня спросят,когда меня т...
33595    у игры множество технических проблем. постоянн...
33596    не ну я конечно подазревал что игра будет мягк...
33597    в игре многое поправили. исправили кучу всего ...
Name: review, Length: 33699, dtype: object

In [114]:
from nltk.corpus import stopwords

nltk.download("stopwords")
stop_words = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vlads\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [115]:
processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
ps = nltk.PorterStemmer()
processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

processed

0        игра, которой победить злодей. отличное карточ...
1        обалденная игра компании. помогает разнообрази...
2        весёлая настольная игра компании, которая раск...
3        общие впечатления игра огонь взрослого. питани...
4        случайно наткнулся игрушку детском мире. учиты...
                               ...                        
33593    песочница котрой развлекаешь сам,в первые долг...
33594    спасибо игру.если спросят,когда тошнило послед...
33595    игры множество технических проблем. постоянные...
33596    подазревал игра мягко говоря очень, столько эт...
33597    игре многое поправили. исправили кучу того, мо...
Name: review, Length: 33699, dtype: object

In [116]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r"\w+")
processed = processed.apply(lambda x: ' '.join(tokenizer.tokenize(x)))
processed

0        игра которой победить злодей отличное карточно...
1        обалденная игра компании помогает разнообразит...
2        весёлая настольная игра компании которая раскр...
3        общие впечатления игра огонь взрослого питание...
4        случайно наткнулся игрушку детском мире учитыв...
                               ...                        
33593    песочница котрой развлекаешь сам в первые долг...
33594    спасибо игру если спросят когда тошнило послед...
33595    игры множество технических проблем постоянные ...
33596    подазревал игра мягко говоря очень столько это...
33597    игре многое поправили исправили кучу того моме...
Name: review, Length: 33699, dtype: object

In [117]:
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)

all_words = nltk.FreqDist(all_words)

# Print the result
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))
word_features = [x[0] for x in all_words.most_common(1500)]

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vlads\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Number of words: 82135
Most common words: [('игра', 18058), ('это', 7916), ('10', 7777), ('очень', 5976), ('игру', 4639), ('игры', 4090), ('просто', 3671), ('игре', 2824), ('играть', 2694), ('то', 2501), ('всё', 2071), ('рекомендую', 2040), ('хорошая', 1892), ('стоит', 1825), ('3', 1823)]


In [118]:
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

In [136]:
messages = list(zip(processed, label))

np.random.seed(1)
np.random.shuffle(messages)

# Call find_features function
feature_set = [(find_features(text), label) for (text, label) in messages]

from sklearn.model_selection import train_test_split

training, test = train_test_split(feature_set, test_size=0.25, random_state=1)

print(len(training))
print(len(test))

25274
8425


In [137]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [138]:
test_model = SklearnClassifier(MultinomialNB()).train(training)

In [140]:
test_model.classify(find_features('Мне понравилась игра, довольно неплохая'))

1

In [141]:
from sklearn.metrics import classification_report, confusion_matrix

text_features, labels = zip(*test)
prediction = test_model.classify_many(text_features)

print(classification_report(labels, prediction))

              precision    recall  f1-score   support

           0       0.66      0.58      0.61      1558
           1       0.91      0.93      0.92      6867

    accuracy                           0.87      8425
   macro avg       0.78      0.75      0.77      8425
weighted avg       0.86      0.87      0.86      8425



In [145]:
pd.DataFrame(confusion_matrix(labels, prediction),
             index=[['actual', 'actual'], ['positive', 'negative']],
             columns = [['predicted', 'predicted'], ['positive', 'negative']])

predicted         
                 negative positive
actual negative       898      660
       positive       468     6399

In [143]:
import pickle

with open(
    r"naive_bayes_classifier.pickle", "wb"
) as classifier_file:
    pickle.dump(test_model, classifier_file)

In [144]:
print(word_features)

['игра', 'это', '10', 'очень', 'игру', 'игры', 'просто', 'игре', 'играть', 'то', 'всё', 'рекомендую', 'хорошая', 'стоит', '3', 'говно', '2', 'всем', 'советую', 'отличная', '1', 'симулятор', 'часов', 'сюжет', 'эта', 'время', 'но', 'которые', '5', 'нет', 'что', 'интересная', 'графика', 'ещё', 'вообще', '4', 'её', 'денег', 'нужно', 'поиграть', 'который', 'играл', 'пока', 'игр', 'купил', 'могу', 'хотя', 'супер', 'деньги', 'а', 'да', 'своих', 'сказать', 'за', 'понравилась', 'тебе', 'скайрим', 'мало', 'персонажей', 'из', 'либо', 'лучший', 'времени', 'геймплей', 'мир', 'атмосфера', 'лучшая', 'классная', 'т', 'почему', 'целом', 'интересно', 'оптимизация', 'и', 'так', 'спасибо', 'особенно', 'довольно', 'я', 'одна', 'неё', 'часть', 'делать', 'немного', 'нравится', 'тех', 'героев', 'часа', 'топ', 'минут', 'каждый', 'покупать', 'бесплатно', 'крутая', 'все', 'пару', 'слишком', 'общем', 'которую', 'затягивает', 'норм', 'самое', 'минус', 'пройти', 'действий', 'сразу', 'разработчики', 'хочется', '100'

In [ ]:
# from nltk.classify.scikitlearn import SklearnClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression, SGDClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.svm import SVC
# from sklearn.metrics import classification_report, confusion_matrix

# names = ['K Nearest Neighbors', 'Decision Tree', 'Random Forest', 'Logistic Regression', 'SGD Classifier',
#          'Naive Bayes', 'Support Vector Classifier']

# classifiers = [
#     KNeighborsClassifier(),
#     DecisionTreeClassifier(),
#     RandomForestClassifier(),
#     LogisticRegression(),
#     SGDClassifier(max_iter=100),
#     MultinomialNB(),
#     SVC(kernel='linear')
# ]

# models = zip(names, classifiers)

# for name, model in models:
#     nltk_model = SklearnClassifier(model)
#     nltk_model.train(training)
#     accuracy = nltk.classify.accuracy(nltk_model, test)
#     print("{} model Accuracy: {}".format(name, accuracy))

In [ ]:
# from sklearn.ensemble import VotingClassifier

# # Since VotingClassifier can accept list type of models
# models = list(zip(names, classifiers))

# nltk_ensemble = SklearnClassifier(VotingClassifier(estimators=models, voting='hard', n_jobs=-1))
# nltk_ensemble.train(training)
# accuracy = nltk.classify.accuracy(nltk_ensemble, test)
# print("Voting Classifier model Accuracy: {}".format(accuracy))

In [ ]:
# text_features, labels = zip(*test)
# prediction = nltk_ensemble.classify_many(text_features)

# print(classification_report(labels, prediction))

In [ ]:
# pd.DataFrame( confusion_matrix(labels, prediction),
#              index=[['actual', 'actual'], ['positive', 'negative']],
#              columns = [['predicted', 'predicted'], ['positive', 'negative']])

In [72]:
# test_model = SklearnClassifier(MultinomialNB()).train(training)
# accuracy = nltk.classify.accuracy(test_model, test)
# test_model.classify(find_features())

1